In [1]:
import pandas as pd

In [2]:
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# df.head()

In [5]:
from transformers import BlipProcessor,BlipForConditionalGeneration
import torch
from PIL import Image
import requests

In [6]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model.to(device)

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-23): 24 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (projection): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((1024,),

In [9]:
# image = Image.open("/content/drive/MyDrive/ova_medical_image/OVA_images/OVA_images/518.jpg")
# text = "what is picture about?"
# inputs = processor(images = image,return_tensors = "pt").to(device)

# output = model.generate(**inputs,max_new_tokens = 50)

In [10]:
# output[0]

In [11]:
# captions = processor.decode(output[0],skip_special_tokens = True).strip()

In [12]:
# captions

In [13]:
folder_path = "/content/drive/MyDrive/ova_medical_image/OVA_images/OVA_images"

In [14]:
# image_paths = []
# for images in os.listdir(folder_path):
#   image_paths.append(os.path.join(folder_path,images))

In [15]:
title = []
for filename in os.listdir(folder_path):
  title.append(filename)

In [16]:
folder_path = "/content/drive/MyDrive/ova_medical_image/OVA_images/OVA_images"

In [17]:
captions= []

for filename in os.listdir(folder_path):
  if filename.endswith(".jpg" or "jpeg" or "png"):
    image = Image.open(os.path.join(folder_path,filename))
    inputs = processor(images = image,return_tensors = "pt").to(device)
    output = model.generate(**inputs)
    captions.append(processor.decode(output[0],skip_special_tokens = True).strip())

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [18]:
captions

['arafed knee pain in a man with a knee injury',
 'there is a woman bending over to pick up something from the ground',
 'a woman with a red rash on her neck and neck',
 'a close up of a person with a lot of red spots on their back',
 'woman in white shirt leaning over a toilet with her hand on the tank',
 'arafed man sitting on a bed with his stomach in pain',
 'woman sitting on bed with hand on face and hand on chin',
 'arafed woman sitting on a toilet in a bathroom with her stomach in pain',
 'a close up of a person with a wound on their hand',
 'there is a woman with a hair transplant on her head',
 'someone is using a blood pressure monitor to measure their blood',
 'a man holding his knee in pain with a red spot',
 'arafed woman sitting on a couch with her hands on her head',
 'arafed woman combing her hair with a blue brush',
 'a close up of a person sitting on the ground with their feet on the ground']

In [19]:
df1 = pd.DataFrame()

In [20]:
df1['image'] = pd.Series(title)
df1['captions'] = pd.Series(captions)

In [21]:
df1.head()

,image,captions
0,638.jpg,arafed knee pain in a man with a knee injury
1,692.jpg,there is a woman bending over to pick up somet...
2,620.jpg,a woman with a red rash on her neck and neck
3,697.jpg,a close up of a person with a lot of red spots...
4,594.jpg,woman in white shirt leaning over a toilet wit...


In [22]:
import csv

In [23]:
output_csvfile = "ova_cancer_t.csv"
with open(output_csvfile,'w',newline = '') as csvfile:
  caption_writer = csv.writer(csvfile)
  caption_writer.writerow(["captions","image"])
  caption_writer.writerows(zip(title,captions))

print(f"Captions saved to {output_csvfile}")


Captions saved to ova_cancer_t.csv
